### 08 Agentic RAG Mainルーチン（Clean Architecture）

**`src/` 配下の Clean Architecture モジュールを `DIContainer` 経由で使用する Main ルーチン。**

gpt-oss:20b を使用する構成のため、**Colab GPU は L4 を使用すること。**

#### アーキテクチャ
- **Domain 層**: ドメインモデル、Port（Protocol）、`WorkflowConfig`
- **Use Cases 層**: `AgentWorkflow`（LangGraph）、`DataIngestion`、各ノード
- **Interface Adapters 層**: `OllamaAdapter`、`ChromaDBAdapter`、`RerankerAdapter`、`PDFLoaderAdapter`、`GradioHandler`
- **Infrastructure 層**: `DIContainer`（依存性の組み立て・注入）

#### ワークフロー
1. **task_planning**: ユーザの質問を分析し、サブタスク（目的＋検索クエリ）を作成
2. **doc_search**: ハイブリッド検索（BM25 + ベクトル）+ Reranker
3. **summarize**: 検索結果を要約（Judge の入力コンテキスト削減）
4. **judge**: 情報の十分性を判定。不足なら doc_search に戻る
5. **generate_answer**: 最終回答をストリーミング生成

**必要なライブラリをインストール**

In [ ]:
# Google Colab に必要なライブラリをインストールする。
# NOTE: Colab では uv ではなく pip を使う。
#       uv は依存解決の過程で numpy 等をアップグレードし、
#       プリインストール済みの scipy 等を壊すため。

# fmt: off
pkgs = [
    "ollama", "langchain-ollama",
    "langchain>=1.2.8", "langchain-core>=1.2.8", "langgraph>=1.0.7",
    "markitdown[all]", "chromadb", "rank-bm25",
    "spacy", "sentence-transformers",
    "pydantic>=2.12", "pydantic-settings>=2.13",
    "gradio>=6.0",
]
# fmt: on

%pip install -U -q {" ".join(pkgs)}

In [ ]:
# spaCy 日本語モデルのインストール
!python -m spacy download ja_core_news_sm

**Google Colab に Ollama をセットアップ**
- 詳細は [01_connect_oss_llm.ipynb](01_connect_oss_llm.ipynb) を参照。

In [ ]:
# Ollama のインストール・起動・モデルのダウンロード
import subprocess
import time
import ollama  # type: ignore

!apt-get install -y -qq zstd
!curl -fsSL https://ollama.com/install.sh | sh

process = subprocess.Popen(
    ["ollama", "serve"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL,
)
time.sleep(5)


def ollama_pull(model: str) -> None:
    """Ollama モデルをダウンロードし、進捗をインライン表示する。"""
    for progress in ollama.pull(model, stream=True):
        status = progress.get("status", "")
        total = progress.get("total") or 0
        completed = progress.get("completed") or 0
        if total:
            line = f"{status}: {completed / total:.0%}"
        else:
            line = status
        print(f"\r{line:<60}", end="", flush=True)
    print(f"\n{model}: Done!")


model_name = "gpt-oss:20b"
ollama_pull(model_name)
!ollama show {model_name}

**`src/` を Python パスに追加**

Google Colab では `notebook/` から `src/` 配下のモジュールを import するため、
`sys.path` に `src/` ディレクトリを追加する。

In [ ]:
import sys
from pathlib import Path

# notebook/ の親ディレクトリ（プロジェクトルート）を取得
project_root = Path.cwd().parent
src_path = str(project_root / "src")

if src_path not in sys.path:
    sys.path.insert(0, src_path)

print(f"Project root: {project_root}")
print(f"src path added: {src_path}")

**DIContainer で依存性を組み立て、Gradio UI を起動**

Clean Architecture の Infrastructure 層にある `DIContainer` が、
すべての依存性（LLM・VectorStore・Reranker・DataLoader）を生成し、
コンストラクタインジェクションで `AgentWorkflow` と `GradioHandler` に注入する。

`WorkflowConfig` のハイパーパラメータは、環境変数（`RAG_` プレフィックス）で上書き可能。

In [ ]:
import logging

from domain.config import WorkflowConfig
from infrastructure.di_container import DIContainer

# ログ設定
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(name)s: %(message)s",
)

# 設定を生成（デフォルト値または環境変数から読み込み）
config = WorkflowConfig()

# DI コンテナで全依存性を組み立て
container = DIContainer(config=config)

# Gradio UI を生成・起動
ui = container.create_ui()
demo = ui.launch()
demo.launch(share=True)

In [ ]:
demo.close()